In [1]:
import math

import gensim
from tqdm import tqdm

from UMLS import UMLSMapper, UMLSEvaluator
from embeddings import Embeddings
from evaluate_embeddings import Benchmark

from vectorize_data import lines_from_file, preprocess

# General

In [2]:
embedding_name = "no_prep_vecs"
umls_mapper = UMLSMapper(from_dir='E:/AML4DH-DATA/UMLS')

initialize UMLSMapper...


# Vectorization

In [3]:
cpq_sentences = lines_from_file(path="E:/AML4DH-DATA/CPG-AMIA2020/Plain Text/cpg-sentences.txt")
print(cpq_sentences[:2])

['Helicobacter pylori ist der wesentliche Risikofaktor für das Magenkarzinom.', 'Die H. pylori-Eradikation mit dem Ziel der Magenkarzinomprävention sollte bei den folgenden Risikopersonen durchgeführt werden (siehe Tabelle unten).']


In [4]:
# tokenization
cpq_sentences = [sentence.split() for sentence in cpq_sentences]
print(cpq_sentences[:2])

[['Helicobacter', 'pylori', 'ist', 'der', 'wesentliche', 'Risikofaktor', 'für', 'das', 'Magenkarzinom.'], ['Die', 'H.', 'pylori-Eradikation', 'mit', 'dem', 'Ziel', 'der', 'Magenkarzinomprävention', 'sollte', 'bei', 'den', 'folgenden', 'Risikopersonen', 'durchgeführt', 'werden', '(siehe', 'Tabelle', 'unten).']]


In [5]:
# cpq_sentences = umls_mapper.standardize_documents(cpq_sentences)
cpq_sentences = umls_mapper.replace_documents_with_umls(cpq_sentences)
# cpq_sentences = preprocess(documents=cpq_sentences, lemmatize=True, remove_stopwords=True)
print(cpq_sentences[:2])

[['C0085508', 'pylori', 'ist', 'der', 'wesentliche', 'Risikofaktor', 'für', 'das', 'Magenkarzinom.'], ['Die', 'H.', 'pylori-Eradikation', 'mit', 'dem', 'Ziel', 'der', 'Magenkarzinomprävention', 'sollte', 'bei', 'den', 'folgenden', 'Risikopersonen', 'durchgeführt', 'werden', '(siehe', 'Tabelle', 'unten).']]


In [6]:
vecs = Embeddings.calculate_vectors(cpq_sentences, use_phrases=False)
Embeddings.save_medical(vecs, embedding_name, umls_mapper)

 ... (more hidden) ...


# Benchmarks

In [4]:
vecs = Embeddings.load(path=f"data/{embedding_name}.kv")

load embedding...


In [7]:
for c, v in vecs.most_similar(umls_mapper.umls_dict["Cisplatin"]):
    print(umls_mapper.un_umls(c), v)
concept_vecs = umls_mapper.get_umls_vectors_only(vecs)

Paclitaxel 0.9073265790939331
Carboplatin 0.9063575267791748
Etoposid 0.8836759924888611
Docetaxel 0.8660497665405273
Capecitabin 0.8628433346748352
Oxaliplatin 0.8613467216491699
Bevacizumab 0.8208531737327576
Rituximab 0.8187896609306335
Vinorelbin 0.8170721530914307
Bleomycin 0.8145763874053955


In [8]:
evaluator = UMLSEvaluator(from_dir='E:/AML4DH-DATA/UMLS', vectors=vecs)

initialize UMLSEvaluator...


In [9]:
benchmark = Benchmark(vecs, umls_mapper, evaluator)
benchmark.choi_benchmark()

Pharmacologic Substance: 3.9181550275259163
Disease or Syndrome: 1.8205877458826087
Neoplastic Process: 2.454774462747514
Clinical Drug: 0
Finding: 0.6427943747281554
Injury or Poisoning: 0.08356520320690844


In [20]:
benchmark.all_categories_benchmark()

 ... (more hidden) ...

Organic Chemical: within 0.4047955686860371, out 0.3272494287251759, distance 0.07754613996086124


 ... (more hidden) ...

Pharmacologic Substance: within 0.4028166942319427, out 0.3246679605345326, distance 0.0781487336974101


 ... (more hidden) ...

...

 ... (more hidden) ...

Chemical Viewed Functionally: within 0.2149998595317205, out 0.28677115088016764, distance -0.07177129134844715
Acquired Abnormality: within 0.4173906147480011, out 0.3631991917941964, distance 0.05419142295380469


 ... (more hidden) ...

Laboratory or Test Result: within 0.16570983193814753, out 0.23367634118860464, distance -0.06796650925045711
0.027127933515222327


0.027127933515222327

# Visualization

In [21]:
from whatlies.embedding import Embedding
from whatlies.embeddingset import EmbeddingSet

In [24]:
emb = EmbeddingSet({umls_mapper.un_umls(c, single_return=True): Embedding(umls_mapper.un_umls(c, single_return=True), vecs[c]) for c in vecs.vocab})
# emb = EmbeddingSet({c: Embedding(c, vecs[c]) for c in vecs.vocab})
emb.plot_interactive("Cisplatin","Carboplatin")